In [1]:
"""ALGORITHM STEPS:

1) The first (standing up detection) RF classifier prediction output sets the 'avg_stand' 
feature in the dataframe passed to (2)

2) The main (position state) RF classifier runs and outputs predicted states

3) The "reconcile function" looks through the predicted states and for OCG and YMOUNT looks for any stand up
motions detected in the vicinity. It corrects accordingly

4) The HMM smooths the results to create clean sequences


** In future, step 1 could be replicated with other isolated movements (e.g. bridge, shrimp, sit up)

THOUGHT: is it possible to attempt to detect stand ups using a different time window?
"""

'ALGORITHM STEPS:\n\n1) The first (standing up detection) RF classifier prediction output sets the \'avg_stand\' \nfeature in the dataframe passed to (2)\n\n2) The main (position state) RF classifier runs and outputs predicted states\n\n3) The "reconcile function" looks through the predicted states and for OCG and YMOUNT looks for any stand up\nmotions detected in the vicinity. It corrects accordingly\n\n4) The HMM smooths the results to create clean sequences\n\n\n** In future, step 1 could be replicated with other isolated movements (e.g. bridge, shrimp, sit up)\n\nTHOUGHT: is it possible to attempt to detect stand ups using a different time window?\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from hmmlearn import hmm

from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from utilities import convert_to_words, print_full, get_position_stats, combine_csv, resolve_acc_gyro, blank_filter, concat_data

TIME_SEQUENCE_LENGTH = 50
polynomial_features = PolynomialFeatures(interaction_only=False, include_bias=True, degree=1)

In [3]:
def trial(df_train, test_data):
    """
    Test 1: 1s followed by 3s
    """
    my_test_data = test_data.drop(['avg_stand'], axis=1)
    y = df_train['state'].values
    X = df_train.drop(['avg_stand', 'stand', 'state', 'index'], axis=1)
    if X.isnull().values.any() == False: 

        rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=8, min_samples_split=4,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)

        X = polynomial_features.fit_transform(X)

        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)

    else: 
        print "Found NaN values"

    rf.fit(X_train, y_train)

    p_test_data = polynomial_features.fit_transform(my_test_data)
    rf_pred2 = rf.predict(p_test_data)
    print rf_pred2
    test_data['state'] = rf_pred2
    final_prediction = convert_to_words(rf_pred2)
    print_full(final_prediction)
    get_position_stats(final_prediction)
    return test_data
    #print 'parameter list: {}'.format(polynomial_features.get_params())

In [4]:
def trial_standup(df_train, test_data):
    """
    Test 1: 1s followed by 3s
    """
    y = df_train['avg_stand'].values
    X = df_train.drop(['avg_stand', 'stand', 'state', 'index'], axis=1)
    if X.isnull().values.any() == False: 

        rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=8, min_samples_split=4,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)

        X = polynomial_features.fit_transform(X)

        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)

    else: 
        print "Found NaN values"

    rf.fit(X_train, y_train)

    p_test_data = polynomial_features.fit_transform(test_data)
    print p_test_data
    rf_pred2 = rf.predict(p_test_data)
    print rf_pred2
    
    # Now we have the estimated stand_up values, we use them to create a new feature
    # in the original df
    #rf_pred3 = rf_pred2.astype(int)
    
    test_data['avg_stand'] = rf_pred2
    return test_data

In [5]:
def reconciler(df):
    relevant_df = df[['avg_stand', 'state']]
    #print relevant_df.state
    #print relevant_df.state.shift(0) # does nothing
    #print relevant_df.state.shift(1) # moves forward 1
    #print relevant_df.state.shift(-1) # moves back 1
    # find any instance of a standup
    # return a list of all the states around that standup
    # check if any of those states are "ymount"
    # if they are, change them to "ocg"
    for i in range(-8, 9):
        title = 'shift{}'.format(i)
        relevant_df[title] = relevant_df.state.shift(i).fillna(False)
        
    detected_standup_df = relevant_df.loc[relevant_df['avg_stand'] == 1].apply(lambda x: x.tolist(), axis=1)
    #print detected_standup_df
    
    surrounding_states = []
    surrounding_indexes = []

    for row in detected_standup_df.iterrows():
        index, data = row
        data_list = data.astype(int).tolist()
        data_list.pop(0) # first value is the 'avg_stand' feature which is not what we want
        data_list.pop(1) # 2nd value is the state which is not what we want
        surrounding_states.append(data_list)
        surrounding_indexes.append(index.tolist())

    
    # Now we check the surrounding states to see if there are any "ymount" values we
    # should convert to "ocg"
    new_values = []
    for index, sequence in enumerate(surrounding_states):
        for values in sequence:
            if (values == 0):
                new_values = [5 if x == 0 else x for x in sequence]
                actual_index = surrounding_indexes[index]
                relevant_df = update_df(df, actual_index, new_values)
                
    return relevant_df

In [6]:
def update_df(df, index, new_values, reach=8):
    
    #print new_values # This is the value at the index (i.e. the row when the stand_up event was 1)
    # need to take this list and lay it over the values in the df at that index
    # TODO: catch indexing error
    
    for x in range(0,reach):
        amount = reach - x
        i = index - (amount*20)
        df.loc[i, 'state'] = new_values[x]
    
    for y in range(0,reach):
        amount = reach - y
        i = index + (amount*20)
        df.loc[i, 'state'] = new_values[y+reach]
    
    return df

In [7]:
def root_sum_square(x, y, z):
        sum = ((x**2)+(y**2)+(z**2))
        rss = math.sqrt(sum)
        return rss

def root_mean_square(x, y, z):
        mean = ((x**2)+(y**2)+(z**2))/3
        rss = math.sqrt(mean)
        return rss

def tiltx(x, y, z):
    try:
        prep = (x/(math.sqrt((y**2)+(z**2))))
        tilt = math.atan(prep)
    except ZeroDivisionError:
        tilt = 0
    return tilt

def tilty(x, y, z):
    try:
        prep = (y/(math.sqrt((x**2)+(z**2))))
        tilt = math.atan(prep)
    except ZeroDivisionError:
        tilt = 0
    return tilt
    
def max_min_diff(max, min):
    diff = max - min
    return diff

def magnitude(x, y, z):
    magnitude = x + y + z
    return magnitude

def create_features(df, _window=50, test=False):
    accel_x = df['ACCEL_X'].astype(float)
    accel_y = df['ACCEL_Y'].astype(float)
    accel_z = df['ACCEL_Z'].astype(float)
    gyro_x = df['GYRO_X'].astype(float)
    gyro_y = df['GYRO_Y'].astype(float)
    gyro_z = df['GYRO_Z'].astype(float)
    
    df2 = pd.DataFrame()
    
    # capture tilt here, then average later
    
    df2['tiltx'] = df.apply(lambda x: tiltx(x['ACCEL_X'], x['ACCEL_Y'], x['ACCEL_Z']), axis=1)
    df2['tilty'] = df.apply(lambda x: tilty(x['ACCEL_X'], x['ACCEL_Y'], x['ACCEL_Z']), axis=1)
    
    # Capture stand state here, then average later
    
    if (test==False):
        df2['stand'] = df['stand'].astype(float)
    
    TIME_SEQUENCE_LENGTH = _window
    
    # Basics
    
    df2['ACCEL_X'] = pd.rolling_mean(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['ACCEL_Y'] = pd.rolling_mean(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['ACCEL_Z'] = pd.rolling_mean(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['GYRO_X'] = pd.rolling_mean(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['GYRO_Y'] = pd.rolling_mean(gyro_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['GYRO_Z'] = pd.rolling_mean(gyro_z, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # rolling median

    df2['rolling_median_x'] = pd.rolling_median(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_median_y'] = pd.rolling_median(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_median_z'] = pd.rolling_median(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_median_gx'] = pd.rolling_median(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_median_gy'] = pd.rolling_median(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_median_gz'] = pd.rolling_median(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # rolling max
    
    df2['rolling_max_x'] = pd.rolling_max(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_max_y'] = pd.rolling_max(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_max_z'] = pd.rolling_max(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_max_gx'] = pd.rolling_max(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_max_gy'] = pd.rolling_max(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_max_gz'] = pd.rolling_max(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # rolling min
    
    df2['rolling_min_x'] = pd.rolling_min(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_min_y'] = pd.rolling_min(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_min_z'] = pd.rolling_min(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_min_gx'] = pd.rolling_min(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_min_gy'] = pd.rolling_min(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_min_gz'] = pd.rolling_min(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # rolling sum
    
    df2['rolling_sum_x'] = pd.rolling_sum(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_sum_y'] = pd.rolling_sum(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_sum_z'] = pd.rolling_sum(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_sum_gx'] = pd.rolling_sum(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_sum_gy'] = pd.rolling_sum(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_sum_gz'] = pd.rolling_sum(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # standard deviation
    
    df2['rolling_std_x'] = pd.rolling_std(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_std_y'] = pd.rolling_std(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_std_z'] = pd.rolling_std(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_std_gx'] = pd.rolling_std(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_std_gy'] = pd.rolling_std(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['rolling_std_gz'] = pd.rolling_std(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # Tilt
    df2['avg_tiltx'] = pd.rolling_mean(df2['tiltx'], TIME_SEQUENCE_LENGTH-2, center=True)
    df2['avg_tilty'] = pd.rolling_mean(df2['tilty'], TIME_SEQUENCE_LENGTH-2, center=True)
    
    
    if (test==False):
        # standing up detection
        df2['avg_stand'] = pd.rolling_mean(df2['stand'], TIME_SEQUENCE_LENGTH-2, center=True)
        print df2['avg_stand']

        # round standing up as we need it to be either '0' or '1' for training later
        df2['avg_stand'] = df2['avg_stand'].apply(lambda x: math.ceil(x))

    ol_upper = _window/2
    ol_lower = ol_upper-1
        
    new_df = df2[ol_lower::ol_upper] # 50% overlap with 30
    
    new_df['max_min_x'] = df2.apply(lambda x: max_min_diff(x['rolling_max_x'], x['rolling_min_x']), axis=1)
    new_df['max_min_y'] = df2.apply(lambda x: max_min_diff(x['rolling_max_y'], x['rolling_min_y']), axis=1)
    new_df['max_min_z'] = df2.apply(lambda x: max_min_diff(x['rolling_max_z'], x['rolling_min_z']), axis=1)
    new_df['max_min_gx'] = df2.apply(lambda x: max_min_diff(x['rolling_max_gx'], x['rolling_min_gx']), axis=1)
    new_df['max_min_gy'] = df2.apply(lambda x: max_min_diff(x['rolling_max_gy'], x['rolling_min_gy']), axis=1)
    new_df['max_min_gz'] = df2.apply(lambda x: max_min_diff(x['rolling_max_gz'], x['rolling_min_gz']), axis=1)
                                                                       
    new_df['acc_rss'] = df2.apply(lambda x: root_sum_square(x['ACCEL_X'], x['ACCEL_Y'], x['ACCEL_Z']), axis=1)
    new_df['gyro_rss'] = df2.apply(lambda x: root_sum_square(x['GYRO_X'], x['GYRO_Y'], x['GYRO_Z']), axis=1)
    
    new_df['acc_rms'] = df2.apply(lambda x: root_mean_square(x['ACCEL_X'], x['ACCEL_Y'], x['ACCEL_Z']), axis=1)
    new_df['gyro_rms'] = df2.apply(lambda x: root_mean_square(x['GYRO_X'], x['GYRO_Y'], x['GYRO_Z']), axis=1)
    
    new_df['acc_magnitude'] = df2.apply(lambda x: magnitude(x['ACCEL_X'], x['ACCEL_Y'], x['ACCEL_Z']), axis=1)
    new_df['gyro_magnitude'] = df2.apply(lambda x: magnitude(x['GYRO_X'], x['GYRO_Y'], x['GYRO_Z']), axis=1)
        
    return new_df


# Test method:
# data = np.array([np.mean(training_data.ACCEL_X[0:30]), np.mean(training_data.ACCEL_X[30:45]), np.mean(training_data.ACCEL_X[30:60])])
# desired_df = pd.DataFrame(data, columns=columns)
# print desired_df


In [8]:
def set_state(df, state):
    """set the state for training"""

    if state == 'your_mount':
        df['state'] = 0
    elif state == 'your_side_control':
        df['state'] = 1
    elif state =='your_closed_guard':
        df['state'] = 2
    elif state =='your_back_control':
        df['state'] = 3
    elif state =='opponent_mount_or_sc':
        df['state'] = 4
    elif state =='opponent_closed_guard':
        df['state'] = 5
    elif state == 'opponent_back_control':
        df['state'] = 6
    elif state =='non_jj':
        df['state'] = 7
        
    return df

In [9]:
def set_stand_state(df, stand_state):
    if (stand_state == 1):
        df['stand'] = 1
    else:
        df['stand'] = 0
    
    print df
    return df

In [10]:
def combine_setState_createFeatures(directory, state, window=40, stand=0):
    """
    convenience method to combine three steps in one function:
    (1) combine multiple csv files, (2) set their movement state for training,
    (3) combine to create time sequences and add features
    """
    combined_data = combine_csv(directory)
    combined_data_updated = set_state(combined_data, state)
    combined_data_updated2 = set_stand_state(combined_data_updated, stand)
    feature_training_data = create_features(combined_data_updated2, window)
    ready_training_data = set_state(feature_training_data, state)
    return ready_training_data

In [11]:
def prep(window=40):
    """prepare the raw sensor data"""

    #1 Your mount
    ymount_td = combine_setState_createFeatures('your_mount_raw_data', 'your_mount', window, 0)
    #2 Your side control
    ysc_td = combine_setState_createFeatures('your_side_control_raw_data', 'your_side_control', window, 0)
    #3 Your closed guard
    ycg_td = combine_setState_createFeatures('your_closed_guard_raw_data', 'your_closed_guard', window, 0)
    #4 Your back control
    ybc_td = combine_setState_createFeatures('your_back_control_raw_data', 'your_back_control', window, 0)
    #5 Opponent mount or opponent side control
    omountsc_td = combine_setState_createFeatures('opponent_mount_and_opponent_side_control_raw_data', 'opponent_mount_or_sc', window, 0)
    #6 Opponent closed guard
    ocg_td = combine_setState_createFeatures('opponent_closed_guard_raw_data', 'opponent_closed_guard', window, 0)
    #7 Opponent back control
    obc_td = combine_setState_createFeatures('opponent_back_control_raw_data', 'opponent_back_control', window, 0)
    #8 "Non jiu-jitsu" motion
    nonjj_td = combine_setState_createFeatures('non_jj_raw_data', 'non_jj', window, 0)
    #9 "stand up" motion
    stand_up_td = combine_setState_createFeatures('standing_up_raw_data', 'opponent_closed_guard', window, 1)

    training_data = concat_data([ymount_td, ysc_td, ycg_td, ybc_td, omountsc_td, ocg_td, obc_td, nonjj_td, stand_up_td])
    # remove NaN
    training_data = blank_filter(training_data)
    return training_data

In [12]:
def prep_test(el_file):
    el_file = 'data/test_cases/' + el_file
    df = pd.DataFrame()
    df = pd.read_csv(el_file, index_col=None, header=0)
    df = resolve_acc_gyro(df)
    df = create_features(df, _window=40, test=True)
    test_data = blank_filter(df)

    return test_data

In [13]:
def test_model_stand(df_train):
    """check model accuracy"""

    y = df_train['avg_stand'].values
    X = df_train.drop(['avg_stand', 'stand', 'state', 'index'], axis=1)
    
    if X.isnull().values.any() == False: 

        rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=8, min_samples_split=4,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)
        
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
        
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_scores = cross_validation.cross_val_score(
    rf, X, df_train.state, cv=10, scoring='accuracy')
    
    print 'rf prediction: {}'.format(accuracy_score(y_test, rf_pred))
    print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (rf_scores.mean(), rf_scores.std() * 2))
    
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))
    

In [14]:
def test_model(df_train):
    """check model accuracy"""

    y = df_train['state'].values
    X = df_train.drop(['state', 'index'], axis=1)
    
    if X.isnull().values.any() == False: 

        rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=8, min_samples_split=4,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)
        
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
        
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_scores = cross_validation.cross_val_score(
    rf, X, df_train.state, cv=10, scoring='accuracy')
    
    print 'rf prediction: {}'.format(accuracy_score(y_test, rf_pred))
    print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (rf_scores.mean(), rf_scores.std() * 2))
    
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

In [15]:
training_data40 = prep(40)

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/DIO_YMOUNT.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_CS.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymount.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs2.csv']


/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/pandas/core/common.py:516: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


       index             timestamp_x  ACCEL_X  ACCEL_Y  ACCEL_Z  \
0          0  2016-03-26 13:33:12.67   -0.358    0.636    0.721   
1          1  2016-03-26 13:33:12.67   -0.071   -0.115   -1.280   
2          2  2016-03-26 13:33:12.67    0.333    0.322   -0.766   
3          3  2016-03-26 13:33:12.67    0.015   -0.092   -0.526   
4          4  2016-03-26 13:33:12.67   -0.179   -0.199   -0.809   
5          5  2016-03-26 13:33:12.67   -0.010   -0.193   -1.108   
6          6  2016-03-26 13:33:12.67    0.054   -0.190   -1.247   
7          7  2016-03-26 13:33:12.67   -0.081   -0.420   -1.198   
8          8  2016-03-26 13:33:12.67   -0.299   -0.408   -1.229   
9          9  2016-03-26 13:33:12.67   -0.360   -0.201   -0.894   
10        10  2016-03-26 13:33:12.67   -0.211   -0.048   -0.570   
11        11  2016-03-26 13:33:12.67   -0.288   -0.029   -0.955   
12        12  2016-03-26 13:33:12.67   -0.383   -0.008   -0.971   
13        13  2016-03-26 13:33:12.67   -0.406   -0.162   -0.80

/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/DIO_YSC.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/GL_ysc_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/ysc1.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/ysc2.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/yscUrs.csv']
      index             timestamp_x  ACCEL_X  ACCEL_Y  ACCEL_Z  \
0         0  2016-03-26 13:29:58.62   -0.646   -0.181   -0.770   
1         1  2016-03-26 13:29:58.62   -0.841   -0.167   -0.888   
2         2  2016-03-26 13:29:58.62   -0.788   -0.104   -0.893   
3         3  2016-03-26 13:29:58.62   -0.623   -0.141   -0.765   
4         4  2016-03-26 13:29:58.62   -0.512   -0.071   -0.807   
5         5  2016-03-26 13:29:58.62   -0.574   -0.069   -0.8

In [16]:
test_data1 = prep_test('test1_ymount_ycg.csv')
test_data4 = prep_test('GL_TEST1_CS.csv')
test_data5 = prep_test('GL_TEST2_CS.csv')
test_data6 = prep_test('GL_TEST3_CS_very_still.csv')
test_data7 = prep_test('GL_TEST1_UrsWearing.csv')
test_data8 = prep_test('DIO_YCG_YMOUNT_YSC.csv')
test_data9 = prep_test('DIO_OCG_YCG_YMOUNT_YBC.csv')


/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

Removed 2 NaN rows
Removed 2 NaN rows
Removed 1 NaN rows
Removed 4 NaN rows
Removed 1 NaN rows
Removed 0 NaN rows
Removed 1 NaN rows


In [22]:
# This is a full sequence

test_data100 = prep_test('CS_OCG_STAND_OCG.csv') #ymount predictions
test_data101 = prep_test('URS_OCG_STAND_OCG1.csv')
test_data102 = prep_test('CS_OCG_STAND_OCG2.csv')
test_data103 = prep_test('URS_OCG_STAND_OCG2.csv') #ymount predictions
test_data104 = prep_test('CS_OCG_STAND_OCG3.csv')
test_data105 = prep_test('URS_OCG_STAND_OCG3.csv')

# step 1

# TO DO: BUG: If I move the test data creation out of this cell it causes an error...very odd

foo = trial_standup(training_data40, test_data105)
#print foo

# step 2
# HAVE TO BE CAREFUL WITH CREATING COPIES OF DF vs. actually changing the original - is creating bugs
step2 = trial(training_data40, foo)
print step2[['state', 'avg_stand']]

# step 3

fixed = reconciler(step2)
print fixed.state

/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

Removed 1 NaN rows
Removed 1 NaN rows
Removed 2 NaN rows
Removed 1 NaN rows
Removed 1 NaN rows
Removed 2 NaN rows
[[  1.00000000e+00  -9.39729584e-01   5.07441844e-01 ...,   8.24291405e+00
   -1.11255263e+00  -1.76526316e-01]
 [  1.00000000e+00  -1.06935680e+00  -1.98342590e-01 ...,   5.28861152e+00
   -1.20068421e+00   1.40773947e+01]
 [  1.00000000e+00  -1.02772170e+00   4.50858635e-01 ...,   5.65764290e+00
   -1.12873684e+00  -7.18386842e+00]
 ..., 
 [  1.00000000e+00  -1.54762618e+00  -6.09596205e-03 ...,   1.31511980e+01
   -9.92684211e-01  -3.72913158e+01]
 [  1.00000000e+00  -1.37916720e+00   7.68410949e-02 ...,   6.40724135e+00
   -1.04205263e+00   1.32910789e+01]
 [  1.00000000e+00  -1.44942186e+00   7.01699118e-02 ...,   2.87800927e+00
   -1.12252632e+00   6.65907895e+00]]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
[5 5 5 0 5 5 5 5 5 5 5 7 5 5 5 5 5 5 5 5 5 5 5 5 7 7]
['opponent_closed_guard', 'opponent_closed_gu

/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


19     5
39     5
59     5
79     5
99     5
119    5
139    5
159    5
179    5
199    7
219    5
239    5
259    5
279    5
299    5
319    5
339    5
359    5
379    5
399    5
419    5
439    5
459    5
479    5
499    7
519    7
Name: state, dtype: int64


In [23]:

# TO DO: BUG: If I move the test data creation out of this cell it causes an error...very odd
test_data1000 = prep_test('URS_OCG_STAND_OCG3.csv')
baz = trial_standup(training_data40, test_data1000)
step2again = trial(training_data40, baz)
pre_pre_smooth = step2again['state'].values
pre_pre_smooth_words = convert_to_words(pre_pre_smooth)

pre_smooth = fixed['state'].values
pre_smooth_words = convert_to_words(pre_smooth)

print pre_pre_smooth_words
print pre_smooth_words

/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

Removed 2 NaN rows
[[  1.00000000e+00  -9.39729584e-01   5.07441844e-01 ...,   8.24291405e+00
   -1.11255263e+00  -1.76526316e-01]
 [  1.00000000e+00  -1.06935680e+00  -1.98342590e-01 ...,   5.28861152e+00
   -1.20068421e+00   1.40773947e+01]
 [  1.00000000e+00  -1.02772170e+00   4.50858635e-01 ...,   5.65764290e+00
   -1.12873684e+00  -7.18386842e+00]
 ..., 
 [  1.00000000e+00  -1.54762618e+00  -6.09596205e-03 ...,   1.31511980e+01
   -9.92684211e-01  -3.72913158e+01]
 [  1.00000000e+00  -1.37916720e+00   7.68410949e-02 ...,   6.40724135e+00
   -1.04205263e+00   1.32910789e+01]
 [  1.00000000e+00  -1.44942186e+00   7.01699118e-02 ...,   2.87800927e+00
   -1.12252632e+00   6.65907895e+00]]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
[5 5 5 0 5 5 5 5 5 5 5 7 5 5 5 5 5 0 5 0 5 5 5 5 7 5]
['opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 

In [24]:
n_components = 8 # ('ybc', 'ymount', 'ysc', 'ycg', 'ocg', 'osc_mount', 'obc', 'other')
# n_components = 3
startprob = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.65,]) # users will probably turn on sensor standing
# startprob = np.array([0.34, 0.33, 0.33])

In [25]:
# transmat = np.array([[0.34, 0.33, 0.33], [0.9, 0.05, 0.05], [0.9, 0.05, 0.05]])

"""
probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

transition_probability = {
        'ymt' : {'ymount': 0.800, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.800, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.800, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.001, 'osc_mount': 0.800, 'ocg': 0.050, 'obc': 0.050, 'other': 0.038},
        'ocg' : {'ymount': 0.100, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.800, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.800, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

transmat = np.array([
                    [0.800, 0.050, 0.010, 0.050, 0.001, 0.050, 0.001, 0.038], 
                    [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                    [0.010, 0.050, 0.800, 0.010, 0.050, 0.001, 0.001, 0.078], 
                    [0.050, 0.010, 0.050, 0.800, 0.001, 0.010, 0.001, 0.078],
                    [0.001, 0.050, 0.010, 0.001, 0.800, 0.050, 0.050, 0.038],
                    [0.100, 0.050, 0.010, 0.010, 0.001, 0.800, 0.001, 0.028],
                    [0.010, 0.050, 0.001, 0.010, 0.050, 0.001, 0.800, 0.078],
                    [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                    ])

In [26]:
# emissionprob = np.array([[0.34, 0.33, 0.33], [0.4, 0.55, 0.05], [0.05, 0.55, 0.4]])

"""

probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

emission_probability = {
        'ymt' : {'ymount': 0.500, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.350, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.400, 'ybc': 0.010, 'osc_mount': 0.500, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.600, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.201, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.210, 'ybc': 0.050, 'osc_mount': 0.600, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ocg' : {'ymount': 0.400, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.400, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.110, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.700, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

emissionprob = np.array([
                        [0.500, 0.050, 0.010, 0.050, 0.001, 0.350, 0.001, 0.038], 
                        [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                        [0.010, 0.050, 0.350, 0.010, 0.500, 0.001, 0.001, 0.078], 
                        [0.050, 0.010, 0.050, 0.700, 0.001, 0.010, 0.101, 0.078],
                        [0.001, 0.050, 0.210, 0.050, 0.600, 0.050, 0.001, 0.038],
                        [0.400, 0.050, 0.010, 0.010, 0.001, 0.400, 0.001, 0.028],
                        [0.010, 0.050, 0.001, 0.110, 0.050, 0.001, 0.700, 0.078],
                        [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                        ])


In [27]:
# Hidden Markov Model with multinomial (discrete) emissions
model = hmm.MultinomialHMM(n_components=n_components,
                           n_iter=10,
                           verbose=False)

model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob
# model.n_features = 8

In [28]:
observations = np.array(pre_smooth)

n_samples = len(observations)
data = observations.reshape((n_samples, -1))
print data

[[5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [7]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [7]
 [7]]


In [29]:
print 'TEST 1'

result = model.decode(data, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_pre_smooth)
print 'result accuracy {}%'.format(result[0])
print 'final result: {}'.format(result[1])

result_words = convert_to_words(result[1])
print '====================='
print 'pre pre smooth words: {}'.format(pre_pre_smooth_words)
print '====================='
print 'result words: {}'.format(result_words)


print '\n'
print "pre pre smooth stats (before stand up detection)"
print get_position_stats(pre_pre_smooth_words)


print '\n'
print "pre smooth stats (before HMM)"
print get_position_stats(pre_smooth_words)

print '\n'

print 'result stats'
print get_position_stats(result_words)

print '******************'




TEST 1
pre smooth: [5 5 5 0 5 5 5 5 5 5 5 7 5 5 5 5 5 0 5 0 5 5 5 5 7 5]
result accuracy -37.0232529784%
final result: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 7 7]
pre pre smooth words: ['opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard']
result words: ['opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opp